# SimParam

> The SimParam class is the cornerstone of a simulation, acting as a global container for parameters that are not specific to any single Population but govern the rules of the entire simulation. Just as the Population class holds the state of individuals, SimParam holds the "genetic laws" of the simulated world, such as trait architecture, genetic maps, and SNP chip definitions.



In [1]:
#| default_exp sp

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass
from dataclasses import field
from typing import List, Optional, TYPE_CHECKING

# The TYPE_CHECKING block is still essential for static type analysis.
if TYPE_CHECKING:
    from .population import Population
    from .trait import TraitA, LociMap, TraitCollection

@flax_dataclass(frozen=True)
class SimParam:
    """
    A container for all global simulation parameters.

    --- JAX JIT Compilation Notes ---

    This class is a JAX Pytree, but it contains a mix of dynamic JAX arrays
    and static Python-native objects. When an instance of `SimParam` is
    passed to a JIT-compiled function, the function is compiled specifically
    for the values of the static attributes.

    - **Dynamic Attributes (Tracable)**: These are `jnp.ndarray`s. Their
      values can change between function calls without causing re-compilation.
      Includes: `gen_map`, `centromere`, `founderPop`, `pedigree`, `var_e`.

    - **Static Attributes (Non-Tracable)**: These are all other attributes
      (`int`, `str`, `bool`, `list`, `tuple`, and custom classes). If any of
      these attributes change value between calls to a JIT-compiled function,
      **it will trigger a re-compilation of that function**.

      This is generally desired behavior for simulation parameters, but care
      must be taken not to modify them inside performance-critical loops.
      The static attributes are: `ploidy`, `traits`, `snp_chips`, `sexes`,
      `recomb_params`, `n_threads`, `track_pedigree`, `track_recomb`,
      `last_id`.
    """
    # --- Core Genome Structure ---
    gen_map: jnp.ndarray
    centromere: jnp.ndarray
    ploidy: int

    # --- Genetic Architecture ---
    # **THE FIX**: Use strings for the type hints to avoid the NameError at runtime.
    traits: Optional['TraitCollection'] = None
    snp_chips: List['LociMap'] = field(default_factory=list)

    # --- Population State Reference ---
    # This was already correct.
    founderPop: Optional['Population'] = None

    # --- Simulation Control ---
    sexes: str = "no"
    recomb_params: tuple = (2.6, 0.0, 0.0)
    n_threads: int = 1

    # --- Pedigree & History Tracking ---
    track_pedigree: bool = False
    track_recomb: bool = False
    last_id: int = 0
    pedigree: Optional[jnp.ndarray] = None

    # --- Default Phenotyping Parameters ---
    var_e: Optional[jnp.ndarray] = None

    @property
    def n_chr(self) -> int:
        return self.gen_map.shape[0]

    @property
    def n_loci_per_chr(self) -> jnp.ndarray:
        return jnp.full((self.n_chr,), self.gen_map.shape[1])

    @property
    def n_traits(self) -> int:
        """
        This logic is now correct and relies on the fix above.
        """
        if self.traits is None:
            return 0
        return self.traits.n_traits

    def __repr__(self) -> str:
        return (f"SimParam(nChr={self.n_chr}, nTraits={self.n_traits}, "
                f"ploidy={self.ploidy}, sexes='{self.sexes}')")

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()